In [108]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from vgn.utils.implicit import get_scene_from_mesh_pose_list
import pandas as pd
from vgn.utils.transform import Rotation, Transform
from vgn.grasp import Grasp
import open3d as o3d
from open3d import JVisualizer
import trimesh

In [109]:
root = "data/pile/data_pile_train_random_raw_4M_radomized_views"
df = pd.read_csv(root + "/grasps_with_clouds_world_reduced.csv")
df.head()

,grasp_id,scene_id,qx,qy,qz,qw,x,y,z,width,label
0,0,aaf72810f32b4124982d1f882e8278b1,-0.259643,0.956220,0.083157,-0.106367,0.169562,0.145066,0.106082,0.026765,1
1,1,749fa350f0fe4c7bb856f06341a7bf9c,0.175647,0.946985,0.262594,0.058414,0.201644,0.197040,0.115168,0.029927,1
2,2,749fa350f0fe4c7bb856f06341a7bf9c,0.181635,0.972635,0.140415,0.035674,0.204992,0.205726,0.117181,0.035908,1
3,3,aaf72810f32b4124982d1f882e8278b1,-0.272389,0.958116,-0.057325,-0.067324,0.155547,0.170059,0.112599,0.025924,1
4,4,749fa350f0fe4c7bb856f06341a7bf9c,0.184423,0.982105,0.034722,0.015881,0.207724,0.212804,0.118663,0.035937,1


In [110]:
index = 1000
df_query = df.loc[index]
grasp_id, scene_id = df_query["grasp_id"], df_query["scene_id"]
mesh_file_name = scene_id + ".npz"
grasp_file_name = str(grasp_id)+ ".npz"
mesh_pose_list = np.load(root + f"/mesh_pose_list/{mesh_file_name}", allow_pickle=True)['pc']
grasp_world = np.load(root + f"/grasp_point_clouds_world_reduced/{grasp_file_name}", allow_pickle=True)['pc']

In [111]:
# Obtain point cloud and normals using ground truth meshes
scene_mesh = get_scene_from_mesh_pose_list(mesh_pose_list)
o3d_scene_mesh = scene_mesh.as_open3d
o3d_scene_mesh.compute_vertex_normals()
pc = o3d_scene_mesh.sample_points_uniformly(number_of_points=10000)
pc_world = np.asarray(pc.points)
pc.colors = o3d.utility.Vector3dVector(np.tile(np.array([0, 0, 0]), (np.asarray(pc.points).shape[0], 1)))

In [112]:
# Transform to gripper local frame
max_opening_width  = 0.08
pos = df_query["x":"z"].to_numpy(np.single)
rotation = Rotation.from_quat(df_query["qx":"qw"].to_numpy(np.single))
grasp = Grasp(Transform(rotation, pos), max_opening_width)
grasp_center = grasp.pose.translation

# Unfortunately VGN/GIGA grasps are not in the grasp frame we want (frame similar to PointNetGPD), so we need to transform them
grasp_frame_rot =  grasp.pose.rotation * Rotation.from_euler('Y', np.pi/2) * Rotation.from_euler('Z', np.pi)
grasp_tf = Transform(grasp_frame_rot, grasp_center)

pc_local = grasp_tf.inverse().transform_point(pc_world)
grasp_local = grasp_tf.inverse().transform_point(grasp_world)

In [113]:
# pc = np.asarray(pc.points)
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scene'}, {'type': 'scene'}]])


fig.add_trace(go.Scatter3d(x=pc_world[:,0], y=pc_world[:,1], z=pc_world[:,2],
                                   mode='markers'), row=1, col=1)

fig.add_trace(go.Scatter3d(x=grasp_world[:,0], y=grasp_world[:,1], z=grasp_world[:,2],
                                   mode='markers'), row=1, col=1)

fig.add_trace(go.Scatter3d(x=pc_local[:,0], y=pc_local[:,1], z=pc_local[:,2],
                                   mode='markers'), row=1, col=2)
                        
fig.add_trace(go.Scatter3d(x=grasp_local[:,0], y=grasp_local[:,1], z=grasp_local[:,2],
                                   mode='markers'), row=1, col=2)
fig.show()

In [115]:
# Viz grasps
grasps_scene = trimesh.Scene()
from vgn.utils import visual
from vgn.utils.implicit import as_mesh
g_mesh = visual.grasp2mesh(grasp)
grasps_scene.add_geometry(g_mesh, node_name=f'graspp')
# grasps_scene.show()
# composed_scene = trimesh.Scene([scene_mesh, grasps_scene])
# composed_scene.show()
visualizer = JVisualizer()
o3d_gripper_mesh = as_mesh(grasps_scene).as_open3d
gripper_pc = o3d_gripper_mesh.sample_points_uniformly(number_of_points=3000)
gripper_pc.colors = o3d.utility.Vector3dVector(np.tile(np.array([1, 1, 0]), (np.asarray(gripper_pc.points).shape[0], 1)))

grasp_world_pc = o3d.geometry.PointCloud()
grasp_world_pc.points = o3d.utility.Vector3dVector(grasp_world)
grasp_world_pc.colors = o3d.utility.Vector3dVector(np.tile(np.array([1.0, 0.45, 0.0]), (np.asarray(grasp_world_pc.points).shape[0], 1)))

visualizer.add_geometry(gripper_pc)
visualizer.add_geometry(pc)
visualizer.add_geometry(grasp_world_pc)
visualizer.show()

JVisualizer with 3 geometries